## Analysis of results on CogenEnv

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ../..

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_style('darkgrid')

In [ ]:
def read_results_csv(folder):
    csv_path = os.path.join('logs', 'cogen', 'rllib', folder, 'test_results.csv')
    df = pd.read_csv(csv_path)
    # change index to seed column
    # print(df.head())
    df.set_index('seed', inplace=True)
    # change 'return' column to 'reward'
    df.rename(columns={'return': 'reward'}, inplace=True)
    return df

In [ ]:
csv_path = os.path.join('logs', 'cogen', 'rllib', 'ppo_lr5e-05_seed123', 'test_results.csv')
df = pd.read_csv(csv_path)

Figure out the best seed + learning rate for each of the pre-distribution shift environments

In [ ]:
from pprint import pprint

In [ ]:
returns = []

for seed in ['123', '456', '789']:
    for lr in ['0.0005', '5e-05', '5e-06']:
        filename = f'ppo_lr{lr}_seed{seed}'
        df = read_results_csv(filename)
        if 'return' in df.columns:
            returns.append((f'lr = {lr}, seed = {seed}', df['return'].mean()))
        else:
            returns.append((f'lr = {lr}, seed = {seed}', df['reward'].mean()))
        # print('lr = {}, seed = {}'.format(lr, seed), df['return'].mean())
pprint(sorted(returns, key=lambda x: x[1], reverse=True))

In [ ]:
ds_returns = []

for seed in ['123', '456', '789']:
    for lr in ['0.0005', '5e-05', '5e-06']:
        df = read_results_csv('ppo_lr{}_seed{}_rm300.0'.format(lr, seed))
        ds_returns.append(('lr = {}, seed = {}'.format(lr, seed), df['return'].mean()))
        # print('lr = {}, seed = {}'.format(lr, seed), df['return'].mean())
pprint(sorted(ds_returns, key=lambda x: x[1], reverse=True))

Seed 456 and learning rate 5e-06 is best for both

In [ ]:
df1 = read_results_csv('ppo_lr5e-06_seed456')
# add a column for training environment
df1['Training environment'] = 'No distribution shift'
# add a column for testing environment
df1['Testing environment'] = 'No distribution shift'

df2 = read_results_csv('ppo_lr5e-06_seed456_rm300.0')
# add a column for training environment
df2['Training environment'] = 'Distribution shift'
# add a column for testing environment
df2['Testing environment'] = 'Distribution shift'

df3 = read_results_csv('ppo_lr5e-06_seed456_rm300.0_distshift')
# add a column for training environment
df3['Training environment'] = 'No distribution shift'
# add a column for testing environment
df3['Testing environment'] = 'Distribution shift'

df = pd.concat([df1, df2, df3])

In [ ]:
df.head()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 3), tight_layout=True)
sns.violinplot(data=df, x='Training environment', y='reward', hue='Testing environment', ax=ax,
               hue_order=['No distribution shift', 'Distribution shift'])
ax.set(xlabel='Training environment', ylabel='Reward')
ax.axvline(0.5, linestyle=':', color='black')
fig.savefig('plots/cogen/violinplot.png', bbox_inches='tight', dpi=300)